# Exploratory Data Analysis and Feature Engineering

## Loading the libraries

For this module, you will need to import 

* `numpy` [link to documentation](https://docs.scipy.org/doc/)
* `pandas` [link to documentation](https://pandas.pydata.org/pandas-docs/stable/)
* `matplotlib.pyplot` [link to documentation](https://matplotlib.org/contents.html)
* `seaborn` [link to documentation](https://seaborn.pydata.org)

We will use `sklearn` as well but later and import it then. Remember to use the `?` and the `TAB` to help you with the code!


In [0]:
# Import the libraries


## Loading the data

For this module, you will consider a simple retail dataset consisting of history of purchases done by customers from a variety of country. The dataset is not of exceptional interest but will allow us to simply illustrate some of the standard steps that can be followed when having a first look at a datset.

**TASKS**

Load the dataset `retail_data.csv` with `pandas`. Recall that you need to 

* use the `read_csv()` method
* point to the location of the dataset (in the `data` folder)
* determine a name under which you want to store the resulting data frame (we suggest the name `customers`)
* specify that the `CustomerID` column is the index column using the `index_col` option

Use the `head` method to display the first few lines of the dataset (you can specify how many lines).

In [0]:
# add your code here


### About the dataset

The dataset is based on data from an online retailer selling gifts and is based on a dataset taken from [here](https://archive.ics.uci.edu/ml/datasets/Online+Retail#).

We have taken the original data set and processed it to create a 'profile' for each customer, which includes a number of features including:

* `Country`: The country their purchases were made from.
* `balance`: Amount of money spent at the store (purchases minus refunds).
* `n_orders`: Total number of orders from the online retailer.
* `time_between_orders`: Average time (in days) between orders.
* `max_spent`: Most amount of money customer spent on a single order.

**TASKS**

* Check the dimensionality of the dataset using the `shape` attribute of the data frame.
* You can also retrieve a few *summary statistics* using the `describe` method.

In [0]:
# add your code here


## Exploring the dataset

At this stage you do not really know what is going on in this dataset. 
You need to go beyond the first impression by considering simple questions like:

* How many customers are you dealing with?
* What country spends how much?
* What has been the company's profit during the last year?
* Are there differences between customers that return and those who don't?

**TASKS**

You will go through these questions and learn new tricks as you move along. The first one is easy to answer, you can use the `nunique()` method applied to the index column. You can retrieve the index column using the `index` attribute of the dataframe.

In [0]:
# how many unique customers are there?


The result should match the number you obtained when inspecting the shape of the DataFrame

### Types of features and One-Hot-Encoding

Data can be both 

* on a *continuous* scale: e.g.: the amount of money spent in the store or time between purchases
* on a *discrete* scale: e.g.: the country.

Discrete variables that have a notion of ordering (for instance, a survey that asks your satisfaction on a scale of 1 to 5) are called *ordinal*. Discrete variables that cannot directly be ordered are usually referred to as *categorical* variables (eg: countries or gender).

In the feature engineering step, one typically needs to pay special attention to discrete variables as many models are not equipped to handle this type of data, particularly if they are just categorical.

In the case where categorical features are present, you need to represent them as numerical values. 
A standard approach to do so is the **one-hot encoding**. 
The input in one-hot encoding is the vector of discrete categorical values, and the output is a sparse matrix with 1s and 0s where each column corresponds to one possible value of the feature.
As an example, let's consider the following trivial dataset:

```
Nick, UK
Laura, IT
Massimo, IT
```

In this case, there are two countries `["UK", "IT"]`, the one-hot-encoding would correspond to the table

$
\begin{array}{l|cc}
& \text{UK} & \text{IT} \\\hline
\text{Nick} & 1 & 0\\
\text{Laura} &0& 1\\
\text{Massimo}&0 & 1
\end{array}
$

**TASKS**

In order to do that on the original dataset:

* select the column `Country` and call it `countries`
* using the function `get_dummies()` from `pandas`, apply the one-hot-encoding
* use `head()` to have a look and make sure it all makes sense
* drop the country column from the initial dataset using `drop` or `del` (so that it is not in our way when applying scaling)


In [0]:
# add your code here


We are going to do pre-processing and imputation on the continuous features, so let's keep this separate for now and join it together later.

### Missing values and Imputation

You may have noticed that there are missing values in the data (`NaN`).
It's very important in general to check whether there are any and

* whether these missing values are informative or not
* whether you can replace the missing values in a sensible way or not

**TASKS**

First, check which column has missing values and how many.
For this, use

* the `isnull()` method applied on the data frame, this returns a dataframe similar to the original one but where every entry is just `True` or `False`
* on the resulting dataframe, apply the `sum()` method which will count how many entries of the column are `True`

In [0]:
# add your code here


*What do you think might be the reason why some customers have missing values as their `time_between_orders`?*

In general, for columns with missing values, there are a few choices on how to handle them. 
This process is usually called *imputation*.

#### Imputation 

There are many strategies to help with missing data and they depend on whether the missing data is numeric or categorical. Recall that you can for example

* simply remove rows where there is missing data (e.g. `.dropna()` can achieve this)
* imputing the values with a summary statistic such as mean or median or most frequent value (e.g. `Imputer` from `sklearn` module)
* replace the values with a sensible estimate

What strategy is best for you problem very much depends on the specifics of your dataset. 

In the current case, the missing values are exclusively found in the `time_between_orders` column, so you should have a look at these rows where this occurs to see if we can gain an understanding of what may be causing these missing values.

**TASKS**

* select the customers for which `time_betwee_orders` is null. For this, use `isnull` on the appropriate column and feed it as row indices to the dataframe to retrieve a subdataframe only corresponding to those customers
* check the shape, make sure it worked!
* have a look at the resulting dataframe, can you spot anything strange? 

In [0]:
# extract the customers that have nan values


Note that `n_orders` seems to often be equal to `1`. 
There is a fairly obvious interpretation for those: since they haven't yet come back, there is no "time between orders".

**TASKS**

You can count the number of time specific values of `n_orders` occur by using the `value_counts()` method applied on the series corresponding to `n_orders`. 

In [0]:
# your code here


The overwhelming majority of missing values can therefore be explained by customers that have not returned before.
It is unclear at this point what the remaining 200 are. The dataset is of course fairly artificial so we won't discuss this in too much detail at this point. 

**TASKS**

In this case, we decide to fill the missing values by 365 days by using the `fillna()` function (this is a **fairly reckless** decision but, again, this notebook is more focused on tools and techniques)

* replace the column `time_between_orders` by the same column where the missing values are filled with value 365 using the `fillna()` function applied on the column
* use `head()` to check

In [0]:
# your code here


### Removing Outliers

Outliers are observation that appear extreme relative to the bulk of the data.
Machine Learning techniques can be sensitive to outliers. 

Here you will see how you can get rid of them if that's what you decide to do. 
There are multiple ways to define outliers, one possibility is to consider all points that are more than `k` standard deviations (`sigma`, $\sigma$) away from the mean (`mu`, $\mu$) of the data.

Below you can see a simple function that takes data and a number of standard deviations and filters out everything that doesn't lie in the range $[\mu-k\sigma, \mu+k\sigma]$.

In [0]:
# this function takes a Series and filters out all elements that are outside
# the range [mu-k*sigma , mu+k*sigma]
def remove_outliers(data, k=3):
    mu       = data.mean() # get the mean
    sigma    = data.std()  # get the standard deviation
    filtered = data[(mu - k*sigma < data) & (data < mu + k*sigma)]
    return filtered

**TASKS**

You can `.apply()` this function to your dataframe. In case that the value is declared an outlier, its value is replace by `NaN`, keeping the structure of the `pd.DataFrame` intact.

In [0]:
# add your code here


**TASKS**

Now, remove the lines with `NaN` values (that correspond to lines with outliers). For this, use the `dropna()` method on the dataframe. 

In [0]:
# add your code here


### Scaling

The different numerical features have completely different scales. This becomes even more obvious when considering a boxplot of the features. You will use the `seaborn` wrapper around `matplotlib` that is great for producing clear plots when looking at data. 
Have a look [here](https://stanford.edu/~mwaskom/software/seaborn/examples/index.html) for a gallery of plots possible with `seaborn`.

**TASKS**

* define a figure environment with the `figure()` method of `matplotlib.pylab` (you can pass a figure size)
* use the `boxplot` function of `seaborn` specifying the appropriate dataframe

In [0]:
# Plot a sns.boxplot() of the customer dataframe, but just take the first


You can see that `n_orders` is defined in a much narrower space than `balance` for example. 
If you were to use the data in an unscaled form, the effect of `balance` might be disproportionnaly high and cause a Machine Learning algorithm to underperform. 

To account for this, it is good practice to center and scale your data, so that all the dimensions fall onto a comparable interval.

**TASKS**

* Define a "scaler" using the `StandardScaler` class imported from `sklearn.preprocessing` (you could also use the `MinMaxScaler` though the `StandardScaler` is more common)
* Apply it on the dataframe using the `fit_transform` method 
* Define a new dataframe similar to the original one but with scaled columns (make sure you specify the `columns` and `index` of the new dataframe using the previous dataframe's `columns` and `index)

In [0]:
# add your code here


**TASKS**

Replot the `boxplot` with the scaled data. Observe that now all the features have most of their "mass" (main part of their observed values) in the same range. Note though that scaling does not change the distribution of features and you can still observe that some features are heavily skewed. 

In [0]:
# replot the boxplot with the scaled data


Now that we've done all this pre-processing, it's useful to save the processed dataset for further use. 

**TASKS**

* use the `to_csv()` method on the dataframe
* set the name to `retail_data_postfeng.csv`
* do the same with the `countries` dataframe, call it `retail_data_countries.csv`

In [0]:
# add your code to save the dataframe


## Relationship between input features

An important tool for the exploratory data analysis step is the **scatter plot**. 

This plot helps visualise the relationship in-between two input features. It may also give you a first indication of the Machine Learning model that could be applied and its complexity (linear vs. non-linear). 

Given the small number of features, you can have a look at the `pairplot` of all of the features: a grid where each pair of feature is displayed against the other. This can help seeing the correlations present in your data. 

* use `sns.pairplot` on the DataFrame to visualise this
* can you interpret some of the relations that appear in the grid?

In [0]:
sns.pairplot(customers)